# Project Structure Experiments

This notebook is used for testing and prototyping various project structure and pipeline ideas
before integrating it into the main `src/` Python modules.

**Note:** This is not part of the production code.
Some code cells may be exploratory or not well-documented.

In [1]:
from src.utils.config_utils import read_config, bump_version

In [4]:
config = read_config()

In [3]:
new_version = bump_version()

✅ Config updated and saved to config.yaml
🔄 Version updated: v1 → v2


In [5]:
config

{'version': 'v2',
 'project': {'name': 'speaker_recognition_binary',
  'description': 'A CNN model for binary speaker recognition using mel-spectrogram inputs.',
  'author': 'Muhd Uwais'},
 'model': {'type': 'cnn',
  'input_shape': [64, 188, 1],
  'conv_filters': [32, 64, 128],
  'dense_units': [512],
  'dropout_rate': [0.25, 0.25, 0.3, 0.5],
  'max_pool_sizes': [2, 2, 2],
  'loss': 'binary_crossentropy',
  'optimizer': 'adam'},
 'training': {'batch_size': 32,
  'epochs': 20,
  'validation_split': 0.2,
  'early_stopping': {'enable': True, 'monitor': 'val_loss', 'patience': 5},
  'reduce_lr': {'enable': True,
   'monitor': 'val_loss',
   'factor': 0.5,
   'patience': 3,
   'min_lr': '1e-6'}},
 'paths': {'save_dir': 'models/cnn_speaker_v1'}}

In [19]:
model_config, version = (cfg := read_config())["model"], cfg["version"]

In [20]:
model_config, version

({'type': 'cnn',
  'input_shape': [64, 188, 1],
  'conv_filters': [32, 64, 128],
  'dense_units': [512],
  'dropout_rate': [0.25, 0.25, 0.3, 0.5],
  'max_pool_sizes': [2, 2, 2],
  'loss': 'binary_crossentropy',
  'optimizer': 'adam'},
 'v2')

In [1]:
from src.data.dataset_loader import AudioDatasetLoader
from src.data.audio_augmentor import AudioAugmentor
from src.data.mel_processor import WaveformToMel, MelAugmentor
from src.models.train import Trainer
from src.models.evaluation import evaluate_model

In [2]:
loader = AudioDatasetLoader(file_path="C:\\voice-speaker-binary-classifier\\data")
x_train, x_test, y_train, y_test = loader.load_dataset()

[2025-11-04 23:47:37] [INFO] - Initialized AudioDatasetLoader with path=C:\voice-speaker-binary-classifier\data, sr=16000, batch_size=32
[2025-11-04 23:47:37] [INFO] - Loading dataset from directory... this may take a few seconds.


Found 863 files belonging to 2 classes.


[2025-11-04 23:47:44] [INFO] - Dataset loaded successfully. Converting to NumPy arrays...
[2025-11-04 23:47:45] [INFO] - Collected 27 batches. Concatenating into full arrays...
[2025-11-04 23:47:45] [INFO] - Converting to 26 batches with batch_size of 32
[2025-11-04 23:47:45] [INFO] - Final dataset shape: X=(26, 32, 48000, 1), Y=(26, 32)
[2025-11-04 23:47:45] [INFO] - Splitting dataset into train and test sets (80/20)...
[2025-11-04 23:47:45] [INFO] - x_train: (20, 32, 48000, 1), y_train: (20, 32)
[2025-11-04 23:47:45] [INFO] - x_test: (6, 32, 48000, 1), y_test: (6, 32)


In [4]:
augmentor = AudioAugmentor(sr=16000, batch_size=32)
x_train_aug, y_train_aug = augmentor.run(x_train, y_train, num_aug=2, shuffle=True)

[2025-11-04 23:48:59] [INFO] - Audio Augmentation initialized with sr=16000, batch_size=32
[2025-11-04 23:48:59] [INFO] - 🚀 Starting audio augmentation | num_aug=2, shuffle=True
[2025-11-04 23:48:59] [INFO] - 🔁 Running augmentation round 1/2
[2025-11-04 23:49:06] [INFO] - 🔁 Running augmentation round 2/2
[2025-11-04 23:49:18] [INFO] - 🔀 Shuffled augmented dataset
[2025-11-04 23:49:18] [INFO] - ✅ Augmentation complete. Final dataset shape: (60, 32, 48000, 1), (60, 32)


In [7]:
x_train_aug.shape, y_train_aug.shape

((60, 32, 48000, 1), (60, 32))

In [8]:
mel_processor = WaveformToMel()
x_train_mel, y_train_mel = mel_processor.run(x_train_aug, y_train_aug)
x_test_mel, y_test_mel = mel_processor.run(x_test, y_test)

[2025-11-04 23:49:43] [INFO] - 🎧 Converting waveform batches to mel-spectrograms...
[2025-11-04 23:49:44] [INFO] - Converted batch 1/60
[2025-11-04 23:49:44] [INFO] - Converted batch 2/60
[2025-11-04 23:49:44] [INFO] - Converted batch 3/60
[2025-11-04 23:49:44] [INFO] - Converted batch 4/60
[2025-11-04 23:49:44] [INFO] - Converted batch 5/60
[2025-11-04 23:49:45] [INFO] - Converted batch 6/60
[2025-11-04 23:49:45] [INFO] - Converted batch 7/60
[2025-11-04 23:49:45] [INFO] - Converted batch 8/60
[2025-11-04 23:49:45] [INFO] - Converted batch 9/60
[2025-11-04 23:49:45] [INFO] - Converted batch 10/60
[2025-11-04 23:49:45] [INFO] - Converted batch 11/60
[2025-11-04 23:49:46] [INFO] - Converted batch 12/60
[2025-11-04 23:49:46] [INFO] - Converted batch 13/60
[2025-11-04 23:49:46] [INFO] - Converted batch 14/60
[2025-11-04 23:49:46] [INFO] - Converted batch 15/60
[2025-11-04 23:49:46] [INFO] - Converted batch 16/60
[2025-11-04 23:49:47] [INFO] - Converted batch 17/60
[2025-11-04 23:49:47] [I

In [9]:
x_train_mel.shape, y_train_mel.shape

((60, 32, 64, 188, 1), (60, 32))

In [10]:
mel_augmentor = MelAugmentor()
x_train_mel_aug, y_train_mel_aug = mel_augmentor.run(x_train_mel, y_train_mel, num_aug=2, shuffle=True)

[2025-11-04 23:50:01] [INFO] - 🚀 Starting mel-spectrogram augmentation pipeline...
[2025-11-04 23:50:01] [INFO] - 🔁 Running augmentation round 1/2...
[2025-11-04 23:50:02] [INFO] - 🔁 Running augmentation round 2/2...
[2025-11-04 23:50:03] [INFO] - 🔀 Shuffled augmented dataset.
[2025-11-04 23:50:03] [INFO] - ✅ Augmentation complete! Final shape: (180, 32, 64, 188, 1)


In [11]:
trainer = Trainer()
model, history = trainer.train(x_train_mel_aug, y_train_mel_aug, verbose=1, shuffle=True)

[2025-11-04 23:50:06] [INFO] - Initialized ModelBuilder (version=v1) successfully.
[2025-11-04 23:50:06] [INFO] - Building CNN model dynamically from configuration...
[2025-11-04 23:50:06] [INFO] - Input Shape: (64, 188, 1)


Starting Model Training Process... version: v1


[2025-11-04 23:50:07] [INFO] - Model compiled successfully with optimizer=adam, loss=binary_crossentropy, metrics=['accuracy', 'recall', 'precision']


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d (Conv2D)                 │ (None, 62, 186, 32)    │           320 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d (MaxPooling2D)    │ (None, 31, 93, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 31, 93, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_1 (Conv2D)               │ (None, 29, 91, 64)     │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_1 (MaxPooling2D)  │ (None, 14, 45, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 14, 45, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_2 (Conv2D)               │ (None, 12, 43, 128)    │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_2 (MaxPooling2D)  │ (None, 6, 21, 128)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_2 (Dropout)             │ (None, 6, 21, 128)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 16128)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 512)            │     8,258,048 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_3 (Dropout)             │ (None, 512)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 1)              │           513 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 8,351,233 (31.86 MB)

 Trainable params: 8,351,233 (31.86 MB)

 Non-trainable params: 0 (0.00 B)

Starting Training...
Epoch 1/20
144/144 ━━━━━━━━━━━━━━━━━━━━ 72s 479ms/step - accuracy: 0.6128 - loss: 0.6852 - precision: 0.4531 - recall: 0.0659 - val_accuracy: 0.6623 - val_loss: 0.6104 - val_precision: 0.9143 - val_recall: 0.0766 - learning_rate: 0.0010
Epoch 2/20
 76/144 ━━━━━━━━━━━━━━━━━━━━ 28s 415ms/step - accuracy: 0.7525 - loss: 0.5160 - precision: 0.7438 - recall: 0.5514

KeyboardInterrupt: 

In [ ]:
metrics = evaluate_model(model, history, x_test_mel, y_test_mel)
print(metrics)